# Symmetrische AES-Verschlüsselung
# Encode -- Verschlüsselung

Beispiel, siehe: https://qvault.io/cryptography/aes-256-cipher-python-cryptography-examples/

In [1]:
from base64 import b64encode, b64decode
import hashlib
from Cryptodome.Cipher import AES
import os
from Cryptodome.Random import get_random_bytes
import json

In [2]:
# Eingabe:
# Klartextdatei:
nameKlartextdatei = 'AESKlartext.txt'
# Passwort:
#password = input("Password: ")
password = "test"

In [3]:
# Test aus Klartextdatei in Variable klartext einlesen
datei = open(nameKlartextdatei,"r")
klartext = datei.read()
datei.close()

In [4]:
# AES 256 encryption/decryption using pycryptodome library

def encrypt(plain_text, password):
    # generate a random salt
    salt = get_random_bytes(AES.block_size)

    # use the Scrypt KDF to get a private key from the password
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create cipher config
    cipher_config = AES.new(private_key, AES.MODE_GCM)

    # return a dictionary with the encrypted text
    cipher_text, tag = cipher_config.encrypt_and_digest(bytes(plain_text, 'utf-8'))
    return {
        'cipher_text': b64encode(cipher_text).decode('utf-8'),
        'salt': b64encode(salt).decode('utf-8'),
        'nonce': b64encode(cipher_config.nonce).decode('utf-8'),
        'tag': b64encode(tag).decode('utf-8')
    }


def decrypt(enc_dict, password):
    # decode the dictionary entries from base64
    salt = b64decode(enc_dict['salt'])
    cipher_text = b64decode(enc_dict['cipher_text'])
    nonce = b64decode(enc_dict['nonce'])
    tag = b64decode(enc_dict['tag'])
    

    # generate the private key from the password and salt
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create the cipher config
    cipher = AES.new(private_key, AES.MODE_GCM, nonce=nonce)

    # decrypt the cipher text
    decrypted = cipher.decrypt_and_verify(cipher_text, tag)

    return decrypted

# First let us encrypt secret message
encrypted = encrypt(klartext, password)
#print(encrypted) # ist dict

In [5]:
# Ausgabe des verschlüsselten Textes in Datei
datei = open('AESgeheimtextAusgabe.txt',"w")
#datei.write(encrypted['cipher_text'])
datei.write(json.dumps(encrypted))
datei.close()

# Ausgabe Passwort:
print("das gewählte Passwort ist: " + password)
# Ausgabe Passwort als Zahl
#Umwandlung str -> bytes -> int, siehe https://www.geeksforgeeks.org/how-to-convert-bytes-to-int-in-python/
PWalsZahl = int.from_bytes(password.encode('utf-8'), "big")
print("gewähltes Passwort als Ganzzahl: " + str(PWalsZahl))

das gewählte Passwort ist: test
gewähltes Passwort als Ganzzahl: 1952805748
